In [ ]:
import pandas as pd
import torch
import os
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset

# Set tokenizers parallelism environment variable
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Check GPU availability and print info
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Available GPU count: {device_count}")
    for i in range(device_count):
        print(f"GPU {i} name:", torch.cuda.get_device_name(i))
        print(f"GPU {i} memory:", torch.cuda.get_device_properties(i).total_memory / 1e9, "GB")

# Load test data
test_df = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
test_df = test_df[['id', 'text']].dropna()
print(f"Test data loaded, {len(test_df)} records total")

# Load pretrained model and tokenizer
model_path = "/kaggle/input/deberta_final/pytorch/default/1/deberta_model_persuade_V2"  # Make sure this is your model save path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Detect available device and move model to that device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model loaded and moved to device: {device}")

# Define tokenize function
def tokenize_fn(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=384)

# Process test data
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_fn, batched=True, num_proc=1)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
print("Test data processing complete")

# Generate predictions
print("Starting prediction generation...")
# Define batch size
batch_size = 8
all_predictions = []
all_probs = []

# Batch processing predictions for efficiency
for i in range(0, len(test_dataset), batch_size):
    batch = test_dataset[i:i+batch_size]
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    
    # Don't calculate gradients to speed up inference
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
    # Get logits
    logits = outputs.logits
    
    # Calculate probabilities and predictions
    probs = torch.nn.functional.softmax(logits, dim=-1)
    preds = torch.argmax(logits, dim=-1)
    
    # Move to CPU and convert to numpy
    all_predictions.extend(preds.cpu().numpy())
    all_probs.extend(probs[:, 1].cpu().numpy())  # Take probability of second class (generated text)
    
    # Print progress
    if (i // batch_size) % 10 == 0:
        print(f"Processed {i+len(batch)}/{len(test_dataset)} records")

# Convert to numpy arrays
test_preds = np.array(all_predictions)
predicted_probs = np.array(all_probs)

# Output detailed results
detailed_df = test_df.copy()
detailed_df['generated_pred'] = test_preds
detailed_df.to_csv("/kaggle/working/deberta_detailed_predictions.csv", index=False)
print("Detailed prediction results saved")

# Create submission file
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'generated': predicted_probs
})
submission_df.to_csv("/kaggle/working/submission.csv", index=False)
print("Submission file generated")

# Print statistics
print(f"Number of predicted AI-generated texts: {sum(test_preds)}")
print(f"Number of predicted human-written texts: {len(test_preds) - sum(test_preds)}")
print(f"Percentage of predicted AI-generated texts: {sum(test_preds) / len(test_preds) * 100:.2f}%")